In [1]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import fsolve
from scipy.integrate import quad

In [2]:
## scenario switch
case=3;  # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0; # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [3]:
## parameter values
alpha = 0.85;           # labor share parameter in manufacturing
theta = 4;              # scopevec for comparative advantage
sigma = 1;      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1;  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma;  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar;  #Foreign's elasticity of demand for embodied energy
# beta = 2.274853;
# gamma= 0.784877595;
beta=1.892412;
gamma=0.807998928;
epsilonS = 0.5;  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5;  #Foreign's energy supply elasticity: betastar/(1 - betastar)
varphi = 2

In [4]:
def opt_tax(p, te, varphi, tax_scenario, ParaList, df):
     #solve for equilibrium
    pe = p[0] * p[0]
    tb_mat = p[1:]
    te = 0
    #print('input is:', p)
    #print(te,varphi, tax_scenario, ParaList, df)
    # print('pe='+str(pe))
    if tax_scenario['tax_sce']=='purete':
        print("shouldn't be here")
        return (1,1,1,1)
        
    # if pe+tb_mat[0]-tb_mat[1]<=0 and tax_scenario['tax_sce']=='EP_hybrid':
    #     # print(tb_mat)
    #     # print(pe)
    #     # tb_mat[0]=tb_mat[1]-pe;
    #     tb_mat[1]=pe+tb_mat[0];
    #     te=tb_mat[1];
 
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
    # print(tb_mat)
     ## optimal values
    # jxbar_hat =   (1 - df['jxbar']) ** (-1) / (((1 - df['jxbar']) ** (-1) - 1) + (1 + (1 - alpha) * tb_mat[0]/pe) ** (-theta) * (1 + tb_mat[0]/pe) ** ((1 - alpha) * theta));
    jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
    j0_hat = (pe+tb_mat[0])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta))
    jmbar_hat = 1
    #print('j0 hat is:', j0_hat)
    #print('denom is:', (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta)))
    #print('num is:', (pe+tb_mat[0])**(-(1-alpha)*theta))
    #print('first part is:', df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta))
    
    if tax_scenario['tax_sce']=='Unilateral':
        te=varphi;
        tb_mat[1]=1;
    
    if tax_scenario['tax_sce']=='puretc':
        te=tb_mat[0];
        jxbar_hat = 1;
        jmbar_hat = 1;
        tb_mat[1]=1;

    if tax_scenario['tax_sce']=='puretp':
        te=tb_mat[0];
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
        tb_mat[1]=1;

    
    if tax_scenario['tax_sce']=='EC_hybrid':
        te=varphi;
        jxbar_hat = 1;
        jmbar_hat = 1;
        tb_mat[1]=1;

    
    if tax_scenario['tax_sce']=='PC_hybrid':
        te=tb_mat[0];
        if tb_mat[1]<0:
            tb_mat[1]=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (tb_mat[0] - tb_mat[2] * tb_mat[0])/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + tb_mat[0])/(pe + tb_mat[1] * tb_mat[0])) ** (theta * (1 - alpha)));
    
    
    if tax_scenario['tax_sce']=='EP_hybrid':
        te=tb_mat[1];
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
   
    if tax_scenario['tax_sce']=='EPC_hybrid':
        te=varphi;
        if tb_mat[1]<0:
            tb_mat[1]=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (tb_mat[0] - tb_mat[2] * tb_mat[0])/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + tb_mat[0])/(pe + tb_mat[1] * tb_mat[0])) ** (theta * (1 - alpha)));
    
                
    
    jxbar_prime = jxbar_hat * df['jxbar'];
    jmbar_prime = jmbar_hat * df['jmbar'];
    
    j0_prime = j0_hat * df['jxbar'];

    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1)) 
    
    Bfunvec1_prime = quad(tempFunction,0,j0_prime, args=(theta, sigmastar))[0];
    Bfunvec2_prime = quad(tempFunction,0,jxbar_prime, args=(theta, sigmastar))[0];

    #if te is too large, Home stop producing
    petbte = pe + tb_mat[0] - te;
    z = pe + tb_mat[0] >= te;
    petbte = petbte * z;

    Qe_hat = (petbte) ** epsilonS;
    Qestar_hat = pe ** epsilonSstar;
          
    if logit==1:
        epsilonS=beta*(1-gamma)/(1-gamma+gamma*petbte**beta);
        epsilonSstar=beta*(1-gamma)/(1-gamma+gamma*pe**beta);
        Qe_hat = (petbte)**beta/(1-gamma+gamma*(petbte)**beta);
        Qestar_hat = pe**beta/(1-gamma+gamma*pe**beta);
 
    Qe_prime = df['Qe'] * Qe_hat;
    Qestar_prime = df['Qestar'] * Qestar_hat;
     
    CeHH_hat = (pe + tb_mat[0]) ** (-epsilonD) * jmbar_hat ** (1 + (1 - sigma)/theta);
    CeHH_prime = df['CeHH'] * CeHH_hat;
       
    
    # CeFH_hat = (1 + (1 - sigmastar)/theta) * pe ** (-(1 - alpha) * sigmastar) * (pe + tb_mat[0]) ** (-alpha) * Bfunvec_prime/(df['jxbar'] ** (1 +1/theta)) * (1 - df['jxbar']) ** (sigmastar/theta);
    CeFH1_hat = (pe +tb_mat[0])**(-epsilonDstar) * j0_hat**(1 + (1 - sigmastar)/theta);
    CeFH2_hat = (1 + (1 - sigmastar)/theta) * ((1-df['jxbar'])/df['jxbar'])**(sigmastar/theta) * pe**(-epsilonDstar) * (1 + tb_mat[0]/pe)**(-alpha) * (Bfunvec2_prime - Bfunvec1_prime)/df['jxbar']**(1+(1-sigmastar)/theta);
    CeFH1_prime = df['CeFH'] * CeFH1_hat;
    CeFH2_prime = df['CeFH'] * CeFH2_hat;
    CeFH_hat = CeFH1_hat + CeFH2_hat;
    if tax_scenario['Base']==1:
        CeFH_hat = pe ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        CeFH_hat = (pe + tb_mat[0]) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        CeFH_hat = (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if np.isnan(CeFH_hat)==True:
        CeFH_hat=0
    CeFH_prime =df['CeFH'] * CeFH_hat;
    
    
    CeHF_hat = (pe + tb_mat[0]) ** (-epsilonD);
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        CeHF_hat = (pe) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        CeHF_hat = (pe + tb_mat[1] * tb_mat[0]) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);
    
    CeHF_prime = df['CeHF'] * CeHF_hat;
    
    
    CeFF_prime = df['CeFF'] * ((1 - jxbar_prime)/(1-df['jxbar'])) ** (1 + (1 - sigmastar)/theta) * pe ** (-epsilonDstar);
    
    ##
    VgHH = df['CeHH']/(1 - alpha);
    VgFF = df['CeFF']/(1 - alpha);
    
    VgFH = df['CeFH'] /(1 - alpha);
    # VgFH_prime = VgFH * pe ** ((1 - sigmastar) * (1 - alpha)) * (1 - (1 - jxbar_prime) ** (1 + (1 - sigmastar)/theta))/ (df['jxbar'] * (1 - df['jxbar']) ** ( (1-sigmastar)/theta));
    VgFH1_hat = (pe + tb_mat[0]) * CeFH1_hat;
    VgFH2_hat = pe**(1 - epsilonDstar) * ((1-j0_prime)**(1+(1-sigmastar)/theta) - (1-jxbar_prime)**(1+(1-sigmastar)/theta))/ (df['jxbar']  * (1 - df['jxbar'] )**( (1-sigmastar)/theta));
    VgFH1_prime = VgFH * VgFH1_hat;
    VgFH2_prime = VgFH * VgFH2_hat;
    VgFH_hat = VgFH1_hat + VgFH2_hat;
    if tax_scenario['Base']==1:
        VgFH_hat = pe * CeFH_hat;
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        VgFH_hat = (pe + tb_mat[0]) * CeFH_hat;
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        VgFH_hat = (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) * CeFH_hat;
    if np.isnan(VgFH_hat)==True:
            VgFH_hat=0
    VgFH_prime = VgFH * VgFH_hat;  

    VgHF = df['CeHF']/(1 - alpha);
    VgHF_hat = (pe + alpha * tb_mat[0]) * CeHF_hat;
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        VgHF_hat = pe * CeHF_hat;
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        VgHF_hat = (pe + alpha * tb_mat[1] * tb_mat[0]) * CeHF_hat; 
    VgHF_prime = VgHF * VgHF_hat;
    Ge_prime = CeHH_prime + CeFH_prime;
    Ge_hat = Ge_prime/df['Ge'];
    Gestar_prime = CeFF_prime + CeHF_prime;
    Ce_prime = CeHH_prime + CeHF_prime;
    Ce_hat = Ce_prime/df['Ce'];
    Cestar_prime = CeFF_prime + CeFH_prime;
    Xe = df['Qe'] - df['Ge'];
    Xe_prime = Qe_prime - Ge_prime;
    Xg = VgFH - VgHF;
    Xg_prime = VgFH_prime - VgHF_prime;
    Xestar_prime = Qestar_prime - Gestar_prime;
    Qeworld_prime=Qe_prime+Qestar_prime;
    Geworld_prime=CeHH_prime+CeFH_prime+CeHF_prime+CeFF_prime;
    pai_g = VgFH - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha);
    subsidy_ratio = (tb_mat[0]/pe * (1 - alpha)) / (1 + tb_mat[0]/pe * (1 - alpha));
    
    Ve_prime=(pe+tb_mat[0]) * Ce_prime;
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Ve_prime = (pe+tb_mat[0]) * CeHH_prime + pe * CeHF_prime;
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Ve_prime = (pe+tb_mat[0]) * CeHH_prime + (pe + tb_mat[1]*tb_mat[0]) * CeHF_prime;
    
    Vestar_prime= (pe+tb_mat[0]) * CeFH_prime  + pe * CeFF_prime;
    Vestar_prime= (pe+tb_mat[0]) * CeFH1_prime + pe * CeFH2_prime + pe * CeFF_prime;
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid' :
        Vestar_prime = pe * Cestar_prime;
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Vestar_prime = (pe + tb_mat[0] - tb_mat[2]*tb_mat[0]) * CeFH_prime + pe * CeFF_prime;    
    
    Vg = df['Ce'] /(1-alpha);
    Vg_prime_hat = (pe + tb_mat[0]) * Ce_hat;
    Vg_prime = Vg * Vg_prime_hat;
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Vg_prime = CeHH_prime/(1-alpha) * (pe + tb_mat[0]) + CeHF_prime/(1-alpha) * pe; 
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Vg_prime = CeHH_prime/(1-alpha) * (pe + tb_mat[0]) + CeHF_prime/(1-alpha) * (pe + tb_mat[1] * tb_mat[0]);   
    
    Vgstar = df['Cestar'] /(1-alpha);
    Vgstar_prime = VgFH_prime + CeFF_prime/(1-alpha)* pe;
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid' :
        Vgstar_prime = Cestar_prime/(1-alpha)* pe;
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Vgstar_prime = CeFF_prime/(1-alpha)* pe + CeFH_prime/(1-alpha)* (pe + tb_mat[0] - tb_mat[2]*tb_mat[0]);
    
    Lg = alpha/(1-alpha) * df['Ge'];
    Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * Ge_prime;
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid':
        Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * CeHH_prime + alpha/(1-alpha) * pe * CeFH_prime;  
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * CeHH_prime + alpha/(1-alpha) * (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) * CeFH_prime;    
    
    Lgstar = alpha/(1-alpha) * df['Gestar'];
    Lgstar_prime = alpha/(1-alpha) * (pe+tb_mat[0]) * CeHF_prime +alpha/(1-alpha) * pe * CeFF_prime;
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Lgstar_prime = alpha/(1-alpha) * pe * Gestar_prime;
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Lgstar_prime = alpha/(1-alpha) * (pe + tb_mat[1]*tb_mat[0]) * CeHF_prime + alpha/(1-alpha) * pe * CeFF_prime;

    delta_Le = (epsilonS/(epsilonS + 1)) * df['Qe'] * (petbte**(epsilonS + 1) - 1);
    delta_Lestar = (epsilonSstar/(epsilonSstar + 1)) * df['Qestar'] * (pe**(epsilonSstar + 1) - 1);
    
    def Func(a, beta, gamma):
        return (((1-gamma)*a**beta)/(1-gamma+gamma*a**beta)**2)
    if logit==1:
        delta_Le = beta * df['Qe'] * quad(Func,1,petbte, args=(beta, gamma))[0];
        delta_Lestar = beta * df['Qestar'] * quad(Func,1,pe, args=(beta, gamma))[0];
        
    leakage1 = -(Qestar_prime - df['Qestar'])/(Qeworld_prime - df['Qeworld']);
    leakage2 = -(Gestar_prime - df['Gestar'])/(Qeworld_prime - df['Qeworld']);
    leakage3 = -(Cestar_prime - df['Cestar'])/(Qeworld_prime - df['Qeworld']);   

    # if pe +  tb_mat[0]   <=0:
    #     print('varphi=')
    #     print(varphi)
    #     print('pe=')
    #     print(pe)
    #     print('tb=')
    #     print(tb_mat[0])
    #     tb_mat[0]=-pe+1
    # print('pe=')
    # print(pe)
    # print('tb=')
    # print(tb_mat[0])
    
    if pe<0:
        pe=tb_mat[0]
    #test=math.log(pe)
    # print(test)
    # delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
    #        + sigma/(sigma-1)* (Vg_prime-Vg) + sigmastar/(sigmastar-1)* (Vgstar_prime-Vgstar) \
    #        - varphi * (Qeworld_prime - df['Qeworld']);
     
    delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
           + Vg * (alpha - 1) * math.log(pe + tb_mat[0]) + Vgstar * (1/theta) * math.log(df['jxbar']/j0_prime * (pe+tb_mat[0])**(-(1-alpha)*theta)) \
           - varphi * (Qeworld_prime - df['Qeworld']);

    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EC_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
           + Vg * (alpha - 1) * math.log(pe + tb_mat[0]) + Vgstar * (alpha - 1) * math.log(pe) \
           - varphi * (Qeworld_prime - df['Qeworld']);

    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
               +  Vg * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jmbar']/jmbar_prime)) \
               +  Vgstar * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jxbar']/jxbar_prime)) \
               - varphi * (Qeworld_prime - df['Qeworld']);
    
    # print(tb_mat)
    # print(pe)
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
               +  Vg * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jmbar']/jmbar_prime)) \
               +  Vgstar * ((alpha - 1) * math.log(pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) + 1/theta * math.log(df['jxbar']/jxbar_prime)) \
               - varphi * (Qeworld_prime - df['Qeworld']);

    if tax_scenario['tax_sce'] == 'Unilateral':
        diff = Qe_prime + Qestar_prime - (CeHH_prime + CeFH_prime + CeHF_prime + CeFF_prime);
        pai_g = (pe + tb_mat[0]) * CeFH2_prime / (1 - alpha) - VgFH2_prime
    
        numerator = varphi * epsilonSstar * Qestar_prime - sigmastar * (1-alpha) * pai_g
        denominator = epsilonSstar * Qestar_prime + epsilonDstar * CeFF_prime
        diff1 = tb_mat[0] - numerator/denominator
        
        return diff**2 + diff1**2

In [6]:
def calc_tax(df, tb_mat, te, varphi, tax_scenario, ParaList):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList

    
    if tax_scenario['tax_sce']=='Baseline':
        tb=0
        prop=1
        te=0
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})
    elif tax_scenario['tax_sce']=='PC_hybrid':
        res = minimize(minuswelfare, [0, 1, 1], bounds=[(0,None),(0,1),(0, 1)],method='trust-constr',args=(te, varphi, tax_scenario, ParaList, df));
        tb_mat = res.x;
        tb=tb_mat[0];
        prop=tb_mat[1];   
        prop2=tb_mat[2];
        te=tb
        return pd.Series({'tb': tb, 'prop': prop, 'prop2': prop2,'te': te})
    
    elif tax_scenario['tax_sce']=='EPC_hybrid':
        res = minimize(minuswelfare, [0, 1, 1], bounds=[(0,None),(0,1),(0, 1)],method='trust-constr',args=(te, varphi, tax_scenario, ParaList, df));
        tb_mat = res.x;
        tb=tb_mat[0];
        prop=tb_mat[1];   
        prop2=tb_mat[2];
        te=varphi
        return pd.Series({'tb': tb, 'prop': prop, 'prop2': prop2,'te': te})
  
    elif tax_scenario['tax_sce']=='EP_hybrid':
        res = minimize(minuswelfare, [0, 1], bounds=[(0,None),(0,None)],method='L-BFGS-B', args=(te, varphi, tax_scenario, ParaList, df));
        tb_mat = res.x;
        tb=tb_mat[0];
        te=tb_mat[1];   
        prop=te-tb;
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})
    elif tax_scenario['tax_sce']=='Unilateral' or tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EC_hybrid' :
        bounds = [(0.01, np.inf),(0, np.inf), (0,1), (0,1)]
        data = (te, varphi, tax_scenario, ParaList, df)
        res = minimize(opt_tax, x0 = (1.0,1.0,1.0,1.0), args = data, bounds = bounds)
        pe = res.x[0]
        tb=res.x[1]
        prop=res.x[2]
        obj_val = opt_tax(res.x, *data)
        #print(obj_val)
        if (obj_val > 0.1):
            print("minimization failed", varphi, 'obj is', obj_val)
        #print(res)
        if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='puretp':
            te=tb
        elif tax_scenario['tax_sce']=='Unilateral' or tax_scenario['tax_sce']=='EC_hybrid':
            te=varphi
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})
    elif tax_scenario['tax_sce']=='purete':
        res = minimize(minuswelfare_purete, 0, method='nelder-mead', args=(tb_mat, varphi, tax_scenario, ParaList, df));
        te = res.x[0]
        tb_mat = [0,1];
        tb=tb_mat[0];
        prop=tb_mat[1];  
        return pd.Series({'tb': tb, 'prop': prop, 'te': te})

In [7]:
df = pd.read_csv("../output/BaselineCarbon_2015.csv", index_col=['region_scenario','regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])
df

,,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
region_scenario,regionbase,,,,,,,,,,,,,,,
1,US as Home,4.480045,27.795946,32.275990,4.598403,1.196111,0.421601,26.059874,5.794514,26.481474,5.020004,27.255985,32.275990,32.275990,0.015921,0.793579
2,EU28 as Home,0.935781,31.340202,32.275986,2.950581,1.013552,0.507676,27.804173,3.964133,28.311848,3.458257,28.817724,32.275982,32.275982,0.017932,0.744319
3,OECD37 as Home,8.625495,23.650503,32.276001,11.293668,2.487537,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498
4,World as Home,32.175999,0.100000,32.276001,32.146000,0.030000,0.030000,0.070000,32.175999,0.100000,32.175999,0.100000,32.276001,32.276001,0.300000,0.999068
5,China as Home,7.522744,24.753246,32.275990,7.345464,0.632472,1.935382,22.362669,7.977936,24.298052,9.280846,22.995142,32.275990,32.275990,0.079652,0.920722
6,OECD and China as Home,16.148245,16.127758,32.276001,20.103180,1.655961,1.381915,9.134943,21.759140,10.516858,21.485096,10.790904,32.276001,32.276001,0.131400,0.923896
7,US and EU as Home,5.415825,26.860159,32.275986,7.728876,2.029770,0.749387,21.767950,9.758646,22.517336,8.478263,23.797718,32.275982,32.275982,0.033280,0.792003


In [8]:
tax_scenario= pd.DataFrame({'tax_sce': 'Unilateral', 'Base':0},index=[1])

In [9]:
from Func import *

In [10]:
def cases(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    # varphilist = np.arange (1.6,2.5,0.1) # marginal damages
    varphilist = np.arange (0,20.1,0.2) # marginal damages

    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    #varphilist = [2]
    varphilist = np.arange (0,30,0.1)
    output=[]
    for varphi in varphilist:    
        te = 0; #initial value of extraction tax for iteration
        tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)

        ## calculate for optimal taxes by maximizing welfare
        tax_df=df.apply(calc_tax, axis=1, raw=False, args=(tb_mat, te, varphi, tax_scenario, ParaList))
        
        appended_df = pd.merge(df, tax_df, on=['region_scenario','regionbase'])

        ## back out other optimal values
        output_df = appended_df.apply(callback, axis=1, raw=False, result_type=None, args=(varphi, tax_scenario, ParaList))
        output_df = pd.merge(tax_df, output_df, on=['region_scenario','regionbase'])
        output.append(output_df)
        #print(varphi)

    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb

    print(tax_scenario['tax_sce'])
    return output

In [11]:
output_all=tax_scenario.apply(cases, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))

minimization failed 6.6000000000000005 obj is 39.77117119553139
minimization failed 6.800000000000001 obj is 39.52372795545834
minimization failed 8.1 obj is 42.76405524443063
minimization failed 8.3 obj is 43.15325688613322
minimization failed 8.6 obj is 43.89119871817128
minimization failed 9.1 obj is 41.63513206133227
minimization failed 9.600000000000001 obj is 44.01487188082651


/usr/local/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


minimization failed 11.3 obj is 7.843412687511843
minimization failed 11.600000000000001 obj is 7.825920158303814
minimization failed 11.8 obj is 6.8141800744637155
minimization failed 11.9 obj is 6.662956826936708
minimization failed 12.4 obj is 6.882323477081198
minimization failed 12.5 obj is 6.468507510911442
minimization failed 12.600000000000001 obj is 6.278994798131337
minimization failed 12.700000000000001 obj is 6.040330292816463
minimization failed 13.100000000000001 obj is 5.818438799741569
minimization failed 14.600000000000001 obj is 4.544618399617387


/usr/local/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


minimization failed 15.100000000000001 obj is 3.990034247038112


...

minimization failed 28.200000000000003 obj is 0.22970517600666268
minimization failed 28.200000000000003 obj is 90.00441085809929
minimization failed 28.200000000000003 obj is 1.3299504551548647
minimization failed 28.200000000000003 obj is 37.906946214879966
minimization failed 28.3 obj is 0.27334159209228387
minimization failed 28.3 obj is 90.64966572009914
minimization failed 28.3 obj is 1.4322988425987682
minimization failed 28.3 obj is 38.450077952776454
minimization failed 28.400000000000002 obj is 0.32072496909695564
minimization failed 28.400000000000002 obj is 91.29291177920572
minimization failed 28.400000000000002 obj is 1.5383154468450195
minimization failed 28.400000000000002 obj is 38.99597817770302
minimization failed 28.5 obj is 0.3718452571205789
minimization failed 28.5 obj is 91.93422208675383
minimization failed 28.5 obj is 1.6479885493191235
minimization failed 28.5 obj is 39.544625840586214
minimization failed 28.6 obj is 0.42669250588135066
minimization failed 28

/usr/local/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


minimization failed 29.1 obj is 2.3821739166365505
minimization failed 29.1 obj is 42.893045725887035
minimization failed 29.200000000000003 obj is 0.8334956751826386
minimization failed 29.200000000000003 obj is 96.37439610679192
minimization failed 29.200000000000003 obj is 2.5171250946416497
minimization failed 29.200000000000003 obj is 43.46034706903563
minimization failed 29.3 obj is 0.9141599153424843
minimization failed 29.3 obj is 97.00234214773457
minimization failed 29.3 obj is 2.655643398995714
minimization failed 29.3 obj is 44.030232177468505
minimization failed 29.400000000000002 obj is 0.9984742755894912
minimization failed 29.400000000000002 obj is 97.62883375084823
minimization failed 29.400000000000002 obj is 2.7977181063417667
minimization failed 29.400000000000002 obj is 44.60268118671126
minimization failed 29.5 obj is 1.0864294100198184
minimization failed 29.5 obj is 98.25391039972223
minimization failed 29.5 obj is 2.943338574588549
minimization failed 29.5 obj 

/usr/local/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


minimization failed 29.6 obj is 98.87760955600245
minimization failed 29.6 obj is 3.092494238326969
minimization failed 29.6 obj is 45.75519211373691
minimization failed 29.700000000000003 obj is 1.2732248220107694
minimization failed 29.700000000000003 obj is 99.49996680609351
minimization failed 29.700000000000003 obj is 3.245174604520858
minimization failed 29.700000000000003 obj is 46.33521496688816
minimization failed 29.8 obj is 1.3720466002147982
minimization failed 29.8 obj is 100.1210159960552
minimization failed 29.8 obj is 3.401369248455394
minimization failed 29.8 obj is 46.917723591760556
minimization failed 29.900000000000002 obj is 1.4744721466134838
minimization failed 29.900000000000002 obj is 100.74078935495163
minimization failed 29.900000000000002 obj is 3.561067809923045
minimization failed 29.900000000000002 obj is 47.502698788209045
Unilateral


In [ ]:
output_list=[]
for i in range(1,len(tax_scenario)+1):
    output_list.append(output_all.loc[i])

In [ ]:
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)

In [ ]:
Outcomes.head(50)

In [ ]:
Outcomes.to_csv('../output/temp_min.csv'.format(case), header=True) 